In [1]:
# !pip install map_boxes
# !pip install tqdm
# !pip install pycocotools

In [2]:
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO

In [3]:
GT_JSON = './detection/dataset/train.json'
PRED_CSV = './submission_det2.csv'

# load ground truth
with open(GT_JSON, 'r') as outfile:
    test_anno = (json.load(outfile))

# load prediction
pred_df = pd.read_csv(PRED_CSV)

In [4]:
'''
[
    [file_name 1, confidence_score, x_min, x_max, y_min, y_max], 
    [file_name 2 confidence_score, x_min, x_max, y_min, y_max],
    ,,,
    [file_name , confidence_score, x_min, x_max, y_min, y_max]
]
'''

new_pred = []

file_names = pred_df['image_id'].values.tolist()
bboxes = pred_df['PredictionString'].values.tolist()

'''
TODO create new_pred
'''
for i, bbox in enumerate(bboxes):
    if isinstance(bbox, float):
        print(f'{file_names[i]} empty box')

for file_name, bbox in tqdm(zip(file_names, bboxes)):
    boxes = np.array(str(bbox).split(' '))
    
    if len(boxes) % 6 == 1:
        boxes = boxes[:-1].reshape(-1, 6)
    elif len(boxes) % 6 == 0:
        boxes = boxes.reshape(-1, 6)
    else:
        raise Exception('error', 'invalid box count')
    for box in boxes:
        new_pred.append([file_name, box[0], box[1], float(box[2]), float(box[4]), float(box[3]), float(box[5])])

test/1291.jpg empty box
test/1474.jpg empty box
test/3741.jpg empty box
test/4757.jpg empty box


4871it [00:00, 29592.84it/s]


In [5]:
'''
[
    [file_name 1, confidence_score, x_min, x_max, y_min, y_max], 
    [file_name 2, confidence_score, x_min, x_max, y_min, y_max],
    ,,,
    [file_name , confidence_score, x_min, x_max, y_min, y_max]
]
'''
    
gt = []
    
   
'''
coco.getImgIds(): return image id list
    
coco.loadImgs(image_id): return image_info
    
image_info['file_name']: return file name
   
coco.getAnnIds(imgIds=image_info['id']): return annotation id
    
coco.loadAnns(ann_ids): return annotation information list (annotation_info_list)
    
annotation_info_list[i]['bbox']: return i'th annotation [x_min, y_min, w, h]
    
annotation_info_list[i]['category_id']: return i'th annotation category
    
'''
coco = COCO(GT_JSON)

'''
TODO create gt
'''
for image_id in coco.getImgIds():
        
    image_info = coco.loadImgs(image_id)[0]
    annotation_id = coco.getAnnIds(imgIds=image_info['id'])
    annotation_info_list = coco.loadAnns(annotation_id)
        
    file_name = image_info['file_name']
        
    for annotation in annotation_info_list:
        gt.append([file_name, annotation['category_id'],
                   float(annotation['bbox'][0]),
                   float(annotation['bbox'][0]) + float(annotation['bbox'][2]),
                   float(annotation['bbox'][1]),
                   (float(annotation['bbox'][1]) + float(annotation['bbox'][3]))])

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


In [7]:
# print(gt)

In [9]:
for line in new_pred:
    print(line)

['test/0000.jpg', '7', '0.9947775602340698', 214.10194, 457.66568, 56.64037, 467.7707]
['test/0000.jpg', '7', '0.9945390820503235', 596.04047, 954.25684, 517.02747, 1024.0]
['test/0000.jpg', '7', '0.9941008687019348', 120.31981, 335.90933, 421.70428, 710.687]
['test/0000.jpg', '7', '0.9886385798454285', 648.208, 877.4001, 333.07214, 598.68317]
['test/0000.jpg', '7', '0.9877954721450806', 454.29297, 649.17926, 598.4266, 874.2031]
['test/0000.jpg', '7', '0.9782375693321228', 388.33047, 596.1632, 187.47209, 548.8915]
['test/0000.jpg', '7', '0.9389642477035522', 558.1382, 708.45996, 341.07846, 609.56335]
['test/0000.jpg', '7', '0.9284080862998962', 266.27512, 439.82422, 431.29858, 575.59735]
['test/0000.jpg', '1', '0.9147720336914062', 568.08594, 765.59467, 106.1524, 355.9142]
['test/0000.jpg', '1', '0.853581428527832', 24.869267, 225.75316, 306.82785, 446.76666]
['test/0000.jpg', '1', '0.7357422709465027', 619.217, 751.2681, 158.61421, 343.70926]
['test/0000.jpg', '1', '0.6926519870758057

In [34]:
'''
calculate mAP
'''

'''
Special mission you sould create mean_average_precision_for_boxes function, not library
'''

mean_ap, average_precisions = mean_average_precision_for_boxes(gt, new_pred, iou_threshold=0.5)

print(mean_ap)

Number of files in annotations: 4883
Number of files in predictions: 4867
Unique classes: 10
Detections length: 4867
Annotations length: 4883
0                              | 0.000000 |    3966
1                              | 0.000000 |    6352
2                              | 0.000000 |     897
3                              | 0.000000 |     936
4                              | 0.000000 |     982
5                              | 0.000000 |    2943
6                              | 0.000000 |    1263
7                              | 0.000000 |    5178
8                              | 0.000000 |     159
9                              | 0.000000 |     468
mAP: 0.000000
0.0
